In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-20"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
16464,2024-04-20,Japão B.League,04:05,Sendai,Hokkaido,1.33,3.12,160.5,1.88,1.82,0.0,0.00,0.00,ATjrd5R5,0.751880,0.320513,0.531915,0.549451,0.072393,252.582,126.248772,0.499833,1.2,1.643168,1.369306,432.75,3.258,1.668583,0.512150,-15.0,398.864,362.062073,0.907733,1.8,1.643168,0.912871,291.06,5.158,4.579030,0.887753,-13.0,75,98,5.77,2.97,206.272,356.720,0.568863,0.022933,NaN,0.07,0.014,23.571429,0.473536,0.5,0.026464,-2.03,-0.406,-5.221675,0.214516,0.2,-0.014516
16465,2024-04-20,Japão B.League,03:05,Utsunomiya Brex,Alvark,1.58,2.29,147.5,1.87,1.83,-4.5,2.03,3.10,IgfjfqdI,0.632911,0.436681,0.534759,0.546448,0.069593,88.034,9.608924,0.109150,2.4,1.341641,0.559017,89.91,1.072,0.049193,0.045889,30.0,94.324,11.761681,0.124694,3.0,0.000000,0.000000,113.10,1.224,0.175442,0.143335,60.0,81,78,1.11,1.45,94.496,103.854,0.259455,0.015289,0.294972,0.43,0.086,6.744186,0.896858,1.0,0.103142,1.12,0.224,5.758929,0.795383,0.9,0.104617
16466,2024-04-20,Japão B.League,03:05,Kyoto,Saga,2.20,1.62,154.5,1.87,1.79,0.0,0.00,0.00,2NFL9Oe5,0.454545,0.617284,0.534759,0.558659,0.071829,219.852,62.081183,0.282377,1.2,1.643168,1.369306,212.00,2.562,0.862247,0.336552,-6.0,203.076,123.145620,0.606402,0.6,1.341641,2.236068,399.64,2.464,1.131119,0.459058,-10.0,80,97,2.65,4.12,224.678,277.724,0.214724,0.030912,NaN,-2.71,-0.542,-2.214022,0.375063,0.2,-0.175063,1.71,0.342,1.812865,0.434169,0.6,0.165831
16467,2024-04-20,Japão B.League,03:05,Ibaraki Robots,Chiba,3.53,1.27,170.5,1.84,1.84,0.0,0.00,0.00,lKknePtC,0.283286,0.787402,0.543478,0.543478,0.070688,323.626,215.581618,0.666144,1.2,1.643168,1.369306,175.78,4.536,3.575287,0.788203,-30.0,187.578,117.758193,0.627783,1.2,1.643168,1.369306,174.87,2.378,1.550555,0.652042,24.0,94,67,1.87,2.61,255.576,224.650,0.665859,0.000000,NaN,-1.00,-0.200,-12.650000,0.297158,0.3,0.002842,-2.24,-0.448,-0.602679,0.521405,0.5,-0.021405
16468,2024-04-20,Japão B.League,02:05,Hiroshima D.,Osaka,1.28,3.44,153.5,1.85,1.85,0.0,0.00,0.00,dpFP8rBB,0.781250,0.290698,0.540541,0.540541,0.071948,115.048,45.875487,0.398751,1.8,1.643168,0.912871,116.22,1.410,0.394905,0.280075,47.0,229.742,71.296214,0.310332,1.2,1.643168,1.369306,307.28,2.702,0.883499,0.326980,-25.0,78,92,1.49,3.34,108.484,194.840,0.647182,0.000000,NaN,0.23,0.046,6.086957,0.645125,0.6,-0.045125,-2.31,-0.462,-5.281385,0.384865,0.3,-0.084865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16704,2024-04-20,Áustria Superliga,12:30,Klosterneuburg,Kapfenberg,1.26,3.69,163.5,1.86,1.88,0.0,0.00,0.00,jP90g4V9,0.793651,0.271003,0.537634,0.531915,0.064654,119.396,21.428841,0.179477,2.4,1.341641,0.559017,128.79,1.614,0.541138,0.335277,28.0,132.028,52.597805,0.398384,2.4,1.341641,0.559017,222.68,1.560,0.796649,0.510673,79.0,81,76,1.59,2.93,107.518,133.456,0.694250,0.007563,NaN,-1.06,-0.212,-1.226415,0.802622,0.8,-0.002622,-0.13,-0.026,-103.461538,0.632614,0.8,0.167386
16705,2024-04-20,Áustria Superliga,12:30,Wels,UBSC Graz,1.27,3.62,155.5,1.87,1.79,0.0,0.00,0.00,4UvbbjD6,0.787402,0.276243,0.534759,0.558659,0.063645,132.560,31.723427,0.239314,1.2,1.643168,1.369306,85.88,1.788,0.542374,0.3033

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
16508,07:00,Geórgia Superliga,TSU,Kavkasia,1.56,Back Home
16538,14:30,Bósnia E Herzegovina Prvenstvo Bih,Slavija,Prijedor,1.56,Back Home
16563,11:00,Portugal Lpb,Imortal,Ovarense,2.13,Back Home
